<div style='color: #690027;' markdown="1">
    <h1>Overfitting</h1> 
</div>

<div style='color: #690027;' markdown="1">
    <h2>Overfitting tegengaan</h2> 
</div>

Wanneer een netwerk te gefixeerd geraakt op de training data zal het minder goed presteren op ongeziene data, het netwerk leert dan als het ware de training data van buiten. Dit wordt <b>overfitting</b> genoemd. Wanneer het netwerk nog kan bijleren, door bevoorbeeld meer lagen toe te voegen, spreekt men van <b>underfitting</b>. Volgende figuur stelt beide concepten visueel voor, een diep neuraal netwerk zoekt als het ware een functie die de invoer afbeeld op de juiste uitvoer en die goed genoeg generaliseert.

<img src="../.images/IntroductieDeepLearning/overfitting.jpg"/>

Overfitting is één van de meest voorkomende problemen bij het bouwen van een diep neuraal netwerk, gelukkig bestaan er verschillende technieken om overfitting tegen te gaan.

<b><u>Meer training data</u></b>

Hoe meer training data je hebt hoe minder waarschijnlijk het is dat je model snel zal overfitten. De training data zal bijdragen aan de aanpassing van de gewichten van het netwerk tijdens het leren. Wanneer er meer training data is zal het model dus minder snel al deze data vanbuiten kunnen leren.

<b><u>Simpeler netwerk</u></b>

Een diep netwerk met veel lagen en gewichten zal veel informatie over de training data kunnen opslaan en leidt tot overfitting. Een te simpel netwerk zal dan weer niet genoeg leren uit de data. Het is vaak aangewezen om met een relatief simpel netwerk te starten en geleidelijk extra lagen of dimensies toe te voegen.

<b><u>Data augmentatie</u></b>

Hoe meer training data hoe beter, soms heb je echter niet heel veel training data. Met data augmentatie ga je je beschikbare training data aanpassen zodat het netwerk steeds andere data te zien krijgt. Bij afbeeldingen kan je bijvoorbeeld de afbeelding een bepaald aantal graden draaien of spiegelen over de x of de y-as. Je kan de intensiteit aanpassen, de afbeelding naar links, rechts, boven of onder verschuiven, enzovoort. Volgende afbeelding toont data augmentatie op een afbeelding van een stoma. Er is enkel gespiegeld over de x en y-as en geroteerd. Verschuiven zou hier geen goed idee zijn want het model herkent afbeeldingen met een stoma in het middelpunt.

<img src="../.images/IntroductieDeepLearning/data_augmentatie.jpg" width="300"/>

<b><u>Dropout</u></b>

Een zeer efficiënte en veelgebruikte methode van regularisatie is het toevoegen van dropout. Door dropout toe te voegen zal een willekeurig gedeelte van de uitvoer van een laag herleidt worden tot 0. Door steeds andere uitvoer te kiezen zal het netwerk geen patronen leren die niet belangrijk zijn. Het percentage van de uitvoer dat naar 0 herleid moet worden wordt de <b>dropout rate</b> genoemd, bij de volgende voorbeelden hebben we steeds een dropout rate van 0.5 gekozen. Volgende afbeelding toont hoe dropout werk in een diep neuraal netwerk.

<img src="../.images/IntroductieDeepLearning/dropout.jpg"/>

Als voorbeeld gaan we opnieuw een netwerk samenstellen, deze keer wordt er echter gebruik gemaakt van data augmentatie van de training data en wordt er na elke feedforward laag dropout toegevoegd.

In [ ]:
import imp
with open('../.scripts/diep_neuraal_netwerk.py', 'rb') as fp:
    diep_neuraal_netwerk = imp.load_module('.scripts', fp, '../.scripts/diep_neuraal_netwerk.py', ('.py', 'rb', imp.PY_SOURCE))
diep_neuraal_netwerk.kies_netwerk()

Na het kiezen van de netwerk architectuur en enkele netwerk parameters kunnen we opnieuw het netwerk visualiseren met volgende codecel.

In [ ]:
diep_neuraal_netwerk.toon_netwerk(regularization=True)

<div style='color: #690027;' markdown="1">
    <h2>Resultaten</h2> 
</div>

Overfitting is het best te zien door de grafiek van de loss waarden over de verschillende epochs te bekijken. Wanneer de training loss daalt maar de validatie loss begint te stijgen is er sprake van overfitting. Een netwerk met regularisatie zal echter (normaal gezien) pas later beginnen overfitten of helemaal niet overfitten (de validatie loss blijft ongeveer gelijk). Kies verschillende netwerken door parameters hierboven aan te passen en kijk welke netwerken overfitten en welke niet door volgende codecel uit te voeren (Wanneer je de parameters van het netwerk veranderd moet je de codecel opnieuw uitvoeren).

In [ ]:
print("Gekozen netwerk met regularisatie\n")
diep_neuraal_netwerk.toon_loss(regularization=True)
diep_neuraal_netwerk.toon_loss_grafiek(regularization=True)

print("Gekozen netwerk zonder regularisatie\n")
diep_neuraal_netwerk.toon_loss(regularization=False)
diep_neuraal_netwerk.toon_loss_grafiek(regularization=False)

<div style='color: #690027;' markdown="1">
    <h2>Stomata detectie in een volledige afbeelding</h2> 
</div>

Momenteel kan het getrainde model een classificatie maken tussen 2 klassen ("wel een stoma" of "geen stoma") wanneer het een afbeelding van 120 pixels op 120 pixels krijgt als invoer. We willen echter dat alle stomata in een grote afbeelding (1600px op 1200px) gevonden kunnen worden met ons model. Om dit te verwezelijken maken we gebruik van een <b>sliding window</b>. Dit is een vierkantje van 120px op 120px dat uit de grote afbeelding wordt geknipt en als invoer van het getrainde model wordt gebruikt. Voor dit vierkantje zal het model dan een voorspelling geven waarna het vierkantje naar rechts opschuift met 5 pixels. Wanneer het vierkantje de volledige breedte van de afbeelding heeft doorlopen zal het opnieuw naar de linkerkant verspringen maar dan 5 pixels lager dan de vorige keer. Dit proces herhaald zich tot de volledige afbeelding verwerkt werd met telkens sprongen van 5 pixels. Volgende afbeelding geeft een illustratie.

<img src="../.images/IntroductieDeepLearning/sliding_window.gif" width="300"/>

Voor elke vierkantje uit de grote afbeelding zal er op deze manier een voorspelling gemaakt worden. Vaak zullen er meerdere vierkantjes rond de effectieve stoma een positieve uitvoer geven ("wel een stoma") omdat er slechts met 5 pixels wordt opgeschoven en dus niet veel verschil in de vierkantjes zit. Om deze vele positieve voorspellingen samen te voegen wordt er gebruik gemaakt van <b>clustering</b>. Clustering zal punten die bij elkaar liggen als 1 cluster beschouwen en het middelpunt van deze clusters zijn dan de gevonden stomata. Volgende afbeelding geeft een voorbeeld van clustering waarbij de kleuren de verschillende clusters voorstellen en de grote punten de centers van de clusters.

<img src="../.images/IntroductieDeepLearning/clustering.jpg" width="400"/>

Bij de detectie speelt de <b>threshold</b> ook een rol. Dit is de drempelwaarde die men neemt voor de uitvoer van het model. Als we bijvoorbeeld 0.5 als threshold nemen zal alle uitvoer groter dan 0.5 als "wel een stoma" beschouwd worden en alle uitvoer kleiner dan 0.5 als "geen stoma". 

Een belangrijke afweging die je moet maken bij het kiezen van de threshold is de verhouding tussen <b>precision</b> en <b>recall</b>. 

<ul>
    <li>Precision: het percentage van de gevonden stomata dat ook daadwerkelijk stoma zijn.</li>
    <li>Recall: het percentage van de totale aanwezige stomata op een afbeelding die gevonden werden.</li>
</ul>

Het is duidelijk dat een lage threshold zal zorgen voor een lage precision maar een hoge recall, en andersom, een hoge threshold zal zorgen voor een hoge precision maar lage recall.

Volgende codecel zal voor 3 afbeeldingen de stomata detecteren met het gekozen netwerk. Je kan de threshold (thr) aanpassen en het resultaat interpreteren.

In [ ]:
diep_neuraal_netwerk.vind_stomata(regularization=True)